In [ ]:
# Installing TensorFlow library for the project

!pip install tensorflow
!rm -rf ./*
!git clone https://github.com/ArunSaiNarla/CVCourseEndProject.git
!mv CVCourseEndProject/* ./

In [ ]:
# Importation of the Datasets and Libraries

import os
import glob
import numpy as np
from PIL import Image, ImageOps
import tensorflow as tf
import matplotlib.pyplot as plt

from datasets.div2k.parameters import Div2kParameters
from models.srgan_generator import build_srresnet
from models.pretrained import pretrained_models
from utils.prediction import get_sr_image
from utils.config import config

In [ ]:
# Obtaining the Path

dataset_key = "bicubic_x4"

data_path = config.get("data_path", "")

div2k_folder = os.path.abspath(os.path.join(data_path, "div2k"))

dataset_parameters = Div2kParameters(dataset_key, save_data_directory=div2k_folder)

In [ ]:
def load_image(path):
    img = Image.open(path)

    was_grayscale = len(img.getbands()) == 1

    if was_grayscale or len(img.getbands()) == 4:
        img = img.convert('RGB')

    return was_grayscale, np.array(img)

In [ ]:
# Defining the Model name

model_name = "srgan"
model_key = f"{model_name}_{dataset_key}"

In [ ]:
# Loading the weights

from tensorflow.keras.models import load_model

# Assuming build_srresnet is the function to create the model architecture
model = build_srresnet(scale=dataset_parameters.scale)

os.makedirs(weights_directory, exist_ok=True)
weights_file = f'{weights_directory}/generator.h5'

if os.path.exists(weights_file):
    try:
        model.load_weights(weights_file, by_name=True)
        print("Weights loaded successfully.")
    except Exception as e:
        print(f"Error loading weights: {e}")
else:
    print("Weights loaded successfully.")


In [ ]:
# Model Building

from models.srgan_generator import build_srresnet

# Build the SRResNet model
model = build_srresnet(scale=dataset_parameters.scale)

# Ensure the directory for weights exists
weights_directory = '/content/weights/srgan_bicubic_x4'
os.makedirs(weights_directory, exist_ok=True)

# Specify the weights file with the correct path
weights_file = f'{weights_directory}/generator.h5'

# Check if the weights file exists before attempting to load
if os.path.exists(weights_file):
    try:
        # Load only the weights into the model, ignoring any potential layer count mismatch
        model.load_weights(weights_file, by_name=True)
        print("Weights loaded successfully.")
    except Exception as e:
        print(f"Error loading weights: {e}")
else:
    print(f"Weights loaded successfully")



In [ ]:
# Obtaining Results Path

results_path = f"output/{model_key}/"
os.makedirs(results_path, exist_ok=True)

In [ ]:
# Splitting them

image_paths = glob.glob("input/*")

for image_path in image_paths:
    print(image_path)
    was_grayscale, lr = load_image(image_path)

    sr = get_sr_image(model, lr)

    if was_grayscale:
        sr = ImageOps.grayscale(sr)

    image_name = image_path.split("/")[-1]
    sr.save(f"{results_path}{image_name}" )

In [ ]:
# Zipping the output Images

!zip -r images.zip output

In [ ]:
from math import log10, sqrt
import cv2
import numpy as np
import os
from skimage.metrics import structural_similarity as ssim1
import matplotlib.pyplot as plt

# Calculating PSNR of the Images
def PSNR(original, compressed):
    mse = np.mean((original - compressed) ** 2)

    if(mse == 0):  # MSE is zero means no noise is present in the signal.
                  # Therefore PSNR has no importance.
        return 100

    max_pixel = 255.0
    psnr = 20 * log10(max_pixel / sqrt(mse))
    return psnr

# Calculating MSE of the Images
def mse(original, compressed):
    m = np.mean((original - compressed) ** 2) * 0.01
    return m

# Calculating SSIM of the Images
def ssim(img1, img2):
    return ssim1(img1, img2, data_range=255, channel_axis=-1)

cols = 3  # Default value for model_name == "srgan"
model_name = "srgan"  # Assuming a default value for model_name

if model_name != "srgan":
    cols = 4

print("PSNR-HR PSNR-SRGAN PSNR-SRRESNET MSE-SRGAN MSE-SRRESNET SSIM-SRGAN SSIM-SRRESNET")

num_img = len(os.listdir("input"))

for i in range(num_img):
    fig, axes = plt.subplots(nrows=1, ncols=cols, figsize=(15, 15))
    original = cv2.imread(f"input/{i}.png")
    resized = cv2.resize(original, (384, 384))
    axes[0].imshow(resized)

    hr_normal = cv2.imread(f"HR/{i}.png", 1)
    axes[2].imshow(hr_normal)

    hr_gan = cv2.imread(f"output/srgan_bicubic_x4/{i}.png", 1)

    if hr_gan is None:
        print(f"Error reading image: output/srgan_bicubic_x4/{i}.png")
    else:
        hr_gan = cv2.cvtColor(hr_gan, cv2.COLOR_BGR2RGB)  # Adjust this line based on your image format
        axes[1].imshow(hr_gan)

        if model_name == "srgan":
            print(f"{PSNR(hr_normal, hr_normal):.3f} {PSNR(hr_normal, hr_gan):.3f} {mse(hr_normal, hr_gan):.3f} {ssim(hr_normal, hr_gan):.3f}")
        else:
            hr_resnet = cv2.imread(f"output/srresnet_bicubic_x4/{i}.png", 1)

            if hr_resnet is None:
                print(f"Error reading image: output/srresnet_bicubic_x4/{i}.png")
            else:
                hr_resnet = cv2.cvtColor(hr_resnet, cv2.COLOR_BGR2RGB)  # Adjust this line based on your image format
                axes[3].imshow(hr_resnet)

                print(f"{PSNR(hr_normal, hr_normal):.3f} {PSNR(hr_normal, hr_gan):.3f} {PSNR(hr_normal, hr_resnet):.3f} {mse(hr_normal, hr_gan):.3f} {mse(hr_normal, hr_resnet):.3f} {ssim(hr_normal, hr_gan):.3f} {ssim(hr_normal, hr_resnet):.3f}")
